<a href="https://colab.research.google.com/github/nongiga/NeuroSEED/blob/master/Neural_SEED_smORFs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NeuroSEED in Small Open Reading Frame (smORF) Proteins

## Introduction
Recently, hyperbolic spaces have been shown to better represent data emerging from latent hierarchies in a variety of contexts (Chamberlain et al, 2017). One such hierarchy emerges among genetic sequences, which over the course of evolution, slowly diverge from one another but nevertheless share a common ancestor (Corso et al, 2021). One application that could benefit from hyperbolic embedding is in the detection of small proteins. Small proteins are responsible for a myriad of functions, including cell-cell communication, regulatory networks, and cell cycle (Gray et al, 2022). Small proteins are difficult to identify empirically due to their small size. However, a recent approach identified novel small protein families by detecting groups of many closely-related suspected sequences and using it to train a neural network (Durrant and Bhatt, 2021). Nevertheless, identifying similarity among sequences through distance calculation is a time-consuming task. Here, we embed small protein sequences in the hyperbolic space and use their distances to predict small protein sequences in an improved computational runtime.

# Implementation
## Motivation
Small Open Reading Frames (smORFs) are a diverse class of proteins that are used in cells across an enormous range of functions. They are defined not by their function, but rather by the size of the genetics sequence that encodes them. With existing methods, it is very difficult to accurately classify which smORFs accomplish which functions. In fact, over 90% of the small protein families have no known domain and almost half are not represented in reference genomes (Sberro et al).

Although advances have been made at identifying which sequences can produce smORFs, it is still very difficult to identify in the computationally efficient manner which kind of smORF that a sequence may belong to. This is because existing methods largely use Euclidean or Levenshtein distance calculation between sequences, without accounting for the hierarchical nature that defines classes of smORFs. For example, two related smORFs share a common genetic ancestor, and thus the calculation for the edit distance between them can be simplified drastically. Hyperbolic embeddings have proven to be more efficient at dealing with hierarchical data, and we seek to use these characteristics to our advantage. 

Our goal with this project is to illuminate whether hyperbolic embeddings improve fidelity of clustering, enabling us to better identify the function of smORFs and consequently predict biological function from a sequence.

## Neural Embeddings in Hyperbolic Space
Neural SEED is a framework that embeds genetic sequences in a particular topology from which edit distance can be calculated.  Our method is to train the encoder against known smORF edit distance data calculated using Levenshtein distances in both Euclidean and hyperbolic spaces. In addition, we compare these values with other methods of calculating distance (square, Manhattan, and cosine) to validate our results.

![Cover](https://raw.githubusercontent.com/gcorso/NeuroSEED/master/tutorial/cover.png)
Figure 1: Encoder takes genetic sequences and maps them onto a topology from which edit distance can be calculated (courtesy of Corso et al.)

We hypothesize that hyperbolic space embeddings will allow for more accurate calculation of edit distance because of the hierarchical nature of the underlying sequences. By training an encoder which can quickly and accurately predict distances between smORF sequences, we can more effectively understand the relationship between these sequences via hierarchical clustering. Hyperbolic spaces also have the additional benefit of portraying ancestral branching relationships quite well, which can provide us an understanding of how the evolutionary path to a given sequence may have emerged.

## Datasets Used
We used data from the SmORFinder dataset. This data consists of strings of genome sequences that represent various known, largely unclassified, smORFs. We subsetted the data into 3 datasets: strings_test (containing 100 sequences), strings_subset (containing 7000 sequences), and clean_strings (containing 35000 sequences). We calculated the Levenshtein Distance between each pair of smORF strings within each dataset. We used the Levenshtein Distance as standard in the calculation of distance between genetic strings. One major goal of creating our embedding is to develop an encoder that quick estimates the Levenshtein Distance between two sequences across datasets. The Levenshtein Distance calculation for the strings_test was about 0.0208 seconds, for strings_subset it was about 1150 seconds, and calculation for clean-strings took more than 10 hours.

Post-processed strings_test and strings_subset datasets are both available on our github (clean_strings is too large to host), as well as the embedded data for reference. We roughly separated each dataset into 60% training, 20% validating, and 20% testing. 

## Other Research Considered
Prior research attempts to classify smORFs using a Hidden Markov Model (Durrant and Bhatt, 2021). However, novel testing illustrates that deep learning models have greater accuracy compared to Hidden Markov Models (Durrant and Bhatt, 2021). However, such testing does not use strong data on smORFs (Sberro et al), instead using a simple thresholding function. The SmORFinder dataset is a more complete reflection of existing smORF data and is a much stronger representation of the current field.


# Code Architecture and Implementation
We use the NeuroSEED model as the foundation of this work (Corso et al, 2020). Geomstats is a core component of the code, providing the necessary functions to accurately calculate hyperbolic distance via an implementation of the Poincare Ball. Our model uses a recurrent neural network to produce an encoder that has learned how to accurately estimate the Levenshtein Distance between two given sequences and embed them onto a given topology. The encoder utilizes one of five distance functions (cosine, Manhattan, square, Euclidean, and hyperbolic) to learn how to estimate the Levenshtein Distance between two sequences.

We evaluate the efficacy of the encoder using a root-mean-square error between the embedded distance between two sequences and the real Levenshtein Distance calculated between them. The lower the error, the more accurate the encoder is at estimating Levenshtein Distances.

Our code makes use of functions from the original NeuroSEED model (Corso et al, 2021) in addition to our own auxilliary functions. The full model is hosted at https://github.com/nongiga/NeuroSEED, whereas this notebook only contains a small portion of the code. 

We highly recommend running this code in VSCode for fidelity and ease of running, as the databases we use are quite large and the most accurate database cannot be hosted on GitHub (only locally).

Setting up the required packages. 

In [ ]:
import torch
import os 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

import sys
import pandas as pd
import numpy as np
import keras
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from Levenshtein import distance as levenshtein_distance

from edit_distance.train import load_edit_distance_dataset
from util.data_handling.data_loader import get_dataloaders
from util.ml_and_math.loss_functions import AverageMeter

import numpy as np
import pickle
import pandas as pd
from scipy.stats import mode

from edit_distance.task.dataset_generator_genomic import EditDistanceGenomicDatasetGenerator

import torch
import torch.nn as nn
import numpy as np
from geomstats.geometry.poincare_ball import PoincareBall

numpy_type_map = {
     'float64': torch.DoubleTensor,
     'float32': torch.FloatTensor,
     'float16': torch.HalfTensor,
     'int64': torch.LongTensor,
     'int32': torch.IntTensor,
     'int16': torch.ShortTensor,
     'int8': torch.CharTensor,
     'uint8': torch.ByteTensor,
 }

## Dataset Generation

This portion takes the longest of any part of the runtime. We recommend pulling directly from the GitHub for the strings_subset and strings_test datasets. You will need to run this code to produce the clean_strings dataset and store it locally.

In [ ]:
!wget https://github.com/bhattlab/SupplementaryInformation/raw/master/SmORFinder/datasets.tar.gz
!tar -zxvf datasets.tar.gz -C '/content/'

df = pd.read_csv("/content/datasets/dataset_FINAL.tsv", sep='\t')
df.head()

# save subset of dataset

df_subset=df.sample(10000, random_state=42)
df_subset.to_csv('smorf_subset.csv')

subset_groups={}

subset_groups['clean_strings']=df['smorf'].str.contains(r'^[ACTG]+$', na=False)

smorfams = df.clust[df.clust.str.startswith('smorfam') & df.y.str.fullmatch('positive')]
md, count = mode(smorfams)
subset_groups['largest_group_strings']=df.clust.str.startswith(md[0]) & df.y.str.fullmatch('positive') & subset_groups['clean_strings']

for name, boo_list in subset_groups.items():
    with open('/content/datasets/'+ name+'.txt', 'w') as f_out:
        f_out.writelines("%s\n" % l for l in df[boo_list].smorf.values)


# generate train-test-val splits for select datasets and pickle them
subsets={"strings_test":100,"strings_subset":7000,"clean_strings":35000}
for n, train_size in subsets.items():
    parser = create_parser('/content/datasets/'+n+'.pkl','/content/datasets/clean_strings.txt',  train_size,round(train_size/10),round(train_size/5))
    generate_datasets(parser)

train_size=50
parser=create_parser('/content/datasets/largest_group_strings.pkl','/content/datasets/largest_group_strings.txt',  train_size,round(train_size/10),round(train_size/5))
generate_datasets(parser)

Creating the functions to calculate distances

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from geomstats.geometry.poincare_ball import PoincareBall

def square_distance(t1_emb, t2_emb,scale=1):
    D = t1_emb - t2_emb
    d = torch.sum(D * D, dim=-1)
    return d


def euclidean_distance(t1_emb, t2_emb,scale=1):
    D = t1_emb - t2_emb
    d = torch.norm(D, dim=-1)
    return d


def cosine_distance(t1_emb, t2_emb,scale=1):
    return 1 - nn.functional.cosine_similarity(t1_emb, t2_emb, dim=-1, eps=1e-6)


def manhattan_distance(t1_emb, t2_emb,scale=1):
    D = t1_emb - t2_emb
    d = torch.sum(torch.abs(D), dim=-1)
    return d

def hyperbolic_geomstats_distance(u,v,scale=1):
    return PoincareBall(u.size()[1]).metric.dist(u,v)
    

def hyperbolic_distance(u, v, epsilon=1e-7):  # changed from epsilon=1e-7 to reduce error
    sqdist = torch.sum((u - v) ** 2, dim=-1)
    squnorm = torch.sum(u ** 2, dim=-1)
    sqvnorm = torch.sum(v ** 2, dim=-1)
    x = 1 + 2 * sqdist / ((1 - squnorm) * (1 - sqvnorm)) + epsilon
    z = torch.sqrt(x ** 2 - 1)
    return torch.log(x + z)


def hyperbolic_distance_numpy(u, v, epsilon=1e-9):
    sqdist = np.sum((u - v) ** 2, axis=-1)
    squnorm = np.sum(u ** 2, axis=-1)
    sqvnorm = np.sum(v ** 2, axis=-1)
    x = 1 + 2 * sqdist / ((1 - squnorm) * (1 - sqvnorm)) + epsilon
    z = np.sqrt(x ** 2 - 1)
    return np.log(x + z)


DISTANCE_TORCH = {
    'square': square_distance,
    'euclidean': euclidean_distance,
    'cosine': cosine_distance,
    'manhattan': manhattan_distance,
    'hyperbolic': hyperbolic_distance
}

General training and evaluation routines used to train the models:

In [ ]:
import argparse
import os
import pickle
import sys
import time
from types import SimpleNamespace

import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from edit_distance.task.dataset import EditDistanceDatasetSampled, EditDistanceDatasetComplete,EditDistanceDatasetSampledCalculated
from edit_distance.task.dataset import EditDistanceDatasetCompleteCalculated
from edit_distance.models.hyperbolics import RAdam
from edit_distance.models.pair_encoder import PairEmbeddingDistance
from util.data_handling.data_loader import get_dataloaders
from util.ml_and_math.loss_functions import MAPE
from util.ml_and_math.loss_functions import AverageMeter


def general_arg_parser():
    """ Parsing of parameters common to all the different models """
    parser = argparse.ArgumentParser()
    parser.add_argument('--data', type=str, default='../../data/edit_qiita_small.pkl', help='Dataset path')
    parser.add_argument('--no-cuda', action='store_true', default=False, help='Disables CUDA training (GPU)')
    parser.add_argument('--seed', type=int, default=42, help='Random seed')
    parser.add_argument('--epochs', type=int, default=2, help='Number of epochs to train')
    parser.add_argument('--lr', type=float, default=0.001, help='Initial learning rate')
    parser.add_argument('--weight_decay', type=float, default=0.0, help='Weight decay')
    parser.add_argument('--dropout', type=float, default=0.0, help='Dropout rate (1 - keep probability)')
    parser.add_argument('--patience', type=int, default=50, help='Patience')
    parser.add_argument('--print_every', type=int, default=1, help='Print training results every')
    parser.add_argument('--batch_size', type=int, default=128, help='Batch size')
    parser.add_argument('--embedding_size', type=int, default=5, help='Size of embedding')
    parser.add_argument('--distance', type=str, default='hyperbolic', help='Type of distance to use')
    parser.add_argument('--workers', type=int, default=0, help='Number of workers')
    parser.add_argument('--loss', type=str, default="mse", help='Loss function to use (mse, mape or mae)')
    parser.add_argument('--plot', action='store_true', default=False, help='Plot real vs predicted distances')
    parser.add_argument('--closest_data_path', type=str, default='', help='Dataset for closest string retrieval tests')
    parser.add_argument('--hierarchical_data_path', type=str, default='', help='Dataset for hierarchical clustering')
    parser.add_argument('--construct_msa_tree', type=str, default='False', help='Whether to construct NJ tree testset')
    parser.add_argument('--extr_data_path', type=str, default='', help='Dataset for further edit distance tests')
    parser.add_argument('--scaling', type=str, default='False', help='Project to hypersphere (for hyperbolic)')
    parser.add_argument('--hyp_optimizer', type=str, default='Adam', help='Optimizer for hyperbolic (Adam or RAdam)')
    return parser


def execute_train(model_class, model_args, args):
    # set device
    args.cuda = not args.no_cuda and torch.cuda.is_available()
    device = 'cpu'
    print('Using device:', device)

    # set the random seed
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.cuda:
        torch.cuda.manual_seed(args.seed)

    # load data
    datasets = load_edit_distance_dataset(args.data)
    loaders = get_dataloaders(datasets, batch_size=args.batch_size, workers=args.workers)

    # fix hyperparameters
    model_args = SimpleNamespace(**model_args)
    model_args.device = device
    model_args.len_sequence = datasets['train'].len_sequence
    model_args.embedding_size = args.embedding_size
    model_args.dropout = args.dropout
    print("Length of sequence", datasets['train'].len_sequence)
    args.scaling = True if args.scaling == 'True' else False

    # generate model
    embedding_model = model_class(**vars(model_args))
    model = PairEmbeddingDistance(embedding_model=embedding_model, distance=args.distance, scaling=args.scaling)
    model.to(device)

    # select optimizer
    if args.distance == 'hyperbolic' and args.hyp_optimizer == 'RAdam':
        optimizer = RAdam(model.parameters(), lr=args.lr)
    else:
        optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)

    # select loss
    loss = None
    if args.loss == "mse":
        loss = nn.MSELoss()
    elif args.loss == "mae":
        loss = nn.L1Loss()
    elif args.loss == "mape":
        loss = MAPE

    # print total number of parameters
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print('Total params', total_params)

    # Train model
    t_total = time.time()
    bad_counter = 0
    best = 1e10
    best_epoch = -1
    start_epoch = 0

    for epoch in range(start_epoch, args.epochs):
        t = time.time()
        loss_train = train(model, loaders['train'], optimizer, loss, device)
        loss_val = test(model, loaders['val'], loss, device)

        # print progress
        if epoch % args.print_every == 0:
            print('Epoch: {:04d}'.format(epoch + 1),
                  'loss_train: {:.6f}'.format(loss_train),
                  'loss_val: {:.6f} MAPE {:.4f}'.format(*loss_val),
                  'time: {:.4f}s'.format(time.time() - t))
            sys.stdout.flush()

        if loss_val[0] < best:
            # save current model
            torch.save(model.state_dict(), '{}.pkl'.format(epoch))
            # remove previous model
            if best_epoch >= 0:
                os.remove('{}.pkl'.format(best_epoch))
            # update training variables
            best = loss_val[0]
            best_epoch = epoch
            bad_counter = 0
        else:
            bad_counter += 1

        if bad_counter == args.patience:
            print('Early stop at epoch {} (no improvement in last {} epochs)'.format(epoch + 1, bad_counter))
            break

    print('Optimization Finished!')
    print('Total time elapsed: {:.4f}s'.format(time.time() - t_total))

    # Restore best model
    print('Loading {}th epoch'.format(best_epoch + 1))
    model.load_state_dict(torch.load('{}.pkl'.format(best_epoch)))

    # Testing
    for dset in loaders.keys():
        if args.plot:
            avg_loss = test_and_plot(model, loaders[dset], loss, device, dset)
        else:
            avg_loss = test(model, loaders[dset], loss, device)
        print('Final results {}: loss = {:.6f}  MAPE {:.4f}'.format(dset, *avg_loss))

    # Nearest neighbour retrieval
    if args.closest_data_path != '':
        print("Closest string retrieval")
        closest_string_testing(encoder_model=model, data_path=args.closest_data_path,
                               batch_size=args.batch_size, device=device, distance=args.distance)

    # Hierarchical clustering
    if args.hierarchical_data_path != '':
        print("Hierarchical clustering")
        hierarchical_clustering_testing(encoder_model=model, data_path=args.hierarchical_data_path,
                                        batch_size=args.batch_size, device=device, distance=args.distance)

    # MSA tree construction on test set
    if args.construct_msa_tree == 'True':
        print("MSA tree construction")
        approximate_guide_trees(encoder_model=model, dataset=datasets['test'],
                                batch_size=args.batch_size, device=device, distance=args.distance)

    # Extra datasets testing (e.g. extrapolation)
    if args.extr_data_path != '':
        print("Extra datasets testing")
        datasets = load_edit_distance_dataset(args.extr_data_path)
        loaders = get_dataloaders(datasets, batch_size=max(1, args.batch_size // 8), workers=args.workers)

        for dset in loaders.keys():
            if args.plot:
                avg_loss = test_and_plot(model, loaders[dset], loss, device, dset)
            else:
                avg_loss = test(model, loaders[dset], loss, device)
            print('Final results {}: loss = {:.6f}  MAPE {:.4f}'.format(dset, *avg_loss))

    torch.save((model_class, model_args, model.embedding_model.state_dict(), args.distance),
               '{}.pkl'.format(model_class.__name__))


def load_edit_distance_dataset(path):
    with open(path, 'rb') as f:
        sequences, distances = pickle.load(f)

    datasets = {}
    for key in sequences.keys():
        if len(sequences[key].shape) == 2:  # datasets without batches
            if key == 'train':
                datasets[key] = EditDistanceDatasetSampled(sequences[key].unsqueeze(0), distances[key].unsqueeze(0),
                                                           multiplicity=10)
            else:
                datasets[key] = EditDistanceDatasetComplete(sequences[key], distances[key])
        else:  # datasets with batches
            datasets[key] = EditDistanceDatasetSampled(sequences[key], distances[key])
    return datasets

def load_edit_distance_dataset_calculate(path):
    with open(path, 'rb') as f:
        sequences, distances = pickle.load(f)

    datasets = {}
    for key in sequences.keys():
        if len(sequences[key].shape) == 2:  # datasets without batches
            if key == 'train':
                datasets[key] = EditDistanceDatasetSampledCalculated(sequences[key].unsqueeze(0), distances[key].unsqueeze(0),
                                                           multiplicity=10)
            else:
                datasets[key] = EditDistanceDatasetCompleteCalculated(sequences[key], distances[key])
        else:  # datasets with batches
            datasets[key] = EditDistanceDatasetSampledCalculated(sequences[key], distances[key])
    return datasets


def train(model, loader, optimizer, loss, device):
    device = 'cpu'
    avg_loss = AverageMeter()
    model.train()

    for sequences, labels in loader:
        # move examples to right device
        # sequences, labels = sequences.to(device), labels.to(device)


        with torch.autograd.set_detect_anomaly(True):
            # forward propagation
            optimizer.zero_grad()
            output = model(sequences)

            # loss and backpropagation
            loss_train = loss(output, labels)
            loss_train.backward()
            optimizer.step()

        # keep track of average loss
        avg_loss.update(loss_train.data.item(), sequences.shape[0])

    return avg_loss.avg


def test(model, loader, loss, device):
    avg_loss = AverageMeter()
    model.eval()

    for sequences, labels in loader:
        # move examples to right device
        # sequences, labels = sequences.to(device), labels.to(device)

        # forward propagation and loss computation
        output = model(sequences)
        loss_val = loss(output, labels).data.item()
        avg_loss.update(loss_val, sequences.shape[0])

    return avg_loss.avg


def test_and_plot(model, loader, loss, device, dataset):
    avg_loss = AverageMeter(len_tuple=2)
    model.eval()

    output_list = []
    labels_list = []

    for sequences, labels in loader:
        # move examples to right device
        sequences, labels = sequences.to(device), labels.to(device)

        # forward propagation and loss computation
        output = model(sequences)
        loss_val = loss[dt](output, labels).data.item()
        mape = MAPE(output, labels).data.item()
        avg_loss.update((loss_val, mape), sequences.shape[0])

        # append real and predicted distances to lists
        output_list.append(output.cpu().detach().numpy())
        labels_list.append(labels.cpu().detach().numpy())

    # save real and predicted distances for offline plotting
    outputs = np.concatenate(output_list, axis=0)
    labels = np.concatenate(labels_list, axis=0)
    pickle.dump((outputs, labels), open(dataset + ".pkl", "wb"))
    # plt.plot(outputs, labels, 'o', color='black')
    # plt.show()

    return avg_loss.avg


Training the models

In [ ]:
#%%
#  Train my models
import os
os.environ['GEOMSTATS_BACKEND'] = 'pytorch'
import torch
from torch import nn
import torch.optim as optim
import time
import argparse

from edit_distance.task.dataset_generator_genomic import EditDistanceGenomicDatasetGenerator
from util.data_handling.data_loader import get_dataloaders
from edit_distance.train import load_edit_distance_dataset,train,test
from edit_distance.models.pair_encoder import PairEmbeddingDistance

class LinearEncoder(nn.Module):
    """  Linear model which simply flattens the sequence and applies a linear transformation. """

    def __init__(self, len_sequence, embedding_size, alphabet_size=4):
        super(LinearEncoder, self).__init__()
        self.encoder = nn.Linear(in_features=alphabet_size * len_sequence, 
                                 out_features=embedding_size)

    def forward(self, sequence):
        # flatten sequence and apply layer
        B = sequence.shape[0]
        sequence = sequence.reshape(B, -1)
        emb = self.encoder(sequence)
        return emb

def run_model(dataset_name, embedding_size, dist_type, string_size, n_epoch):
    device = 'cpu'
    torch.manual_seed(2021)
    if device == 'cuda':
        torch.cuda.manual_seed(2021)

    # load data
    datasets = load_edit_distance_dataset(dataset_name)
    loaders = get_dataloaders(datasets, batch_size=128, workers=0)

    # model, optimizer and loss

    encoder = LinearEncoder(string_size, embedding_size)

    model = PairEmbeddingDistance(embedding_model=encoder, distance=dist_type,scaling=True)
    loss = nn.MSELoss()

    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    optimizer.zero_grad() 


    # training
    for epoch in range(0, n_epoch):
        t = time.time()
        loss_train = train(model, loaders['train'], optimizer, loss, device)
        loss_val = test(model, loaders['val'], loss, device)

        # print progress
        
        if epoch % 5 == 0:
            print('Epoch: {:02d}'.format(epoch),
                'loss_train: {:.6f}'.format(loss_train),
                'loss_val: '.format(loss_val),
                'time: {:.4f}s'.format(time.time() - t))
        
    # testing
    for dset in loaders.keys():
        avg_loss = test(model, loaders[dset], loss, device)
        print('Final results {}: loss = {:.6f}'.format(dset, avg_loss))

    return model, avg_loss

def create_parser(out, source, train,val,test):
    parser = argparse.ArgumentParser()
    parser.add_argument('--out', type=str, default=out, help='Output data path')
    parser.add_argument('--train_size', type=int, default=train, help='Training sequences')
    parser.add_argument('--val_size', type=int, default=val, help='Validation sequences')
    parser.add_argument('--test_size', type=int, default=test, help='Test sequences')
    parser.add_argument('--source_sequences', type=str, default=source, help='Sequences data path')
    return parser


def generate_datasets(parser):
    args, unknown = parser.parse_known_args()
    # load and divide sequences
    with open(args.source_sequences, 'rb') as f:
        L = f.readlines()
    L = [l[:-1].decode('UTF-8') for l in L]

    strings = {
        'train': L[:args.train_size],
        'val': L[args.train_size:args.train_size + args.val_size],
        'test': L[args.train_size + args.val_size:args.train_size + args.val_size + args.test_size]
    }

    data = EditDistanceGenomicDatasetGenerator(strings=strings)
    data.save_as_pickle(args.out)

    return strings

string_size=153
n_epoch = 10
e_size=np.logspace(1,9,num=9-1, base=2,endpoint=False, dtype=int)

# dist_types=['hyperbolic', 'euclidean', 'square', 'manhattan', 'cosine']
dist_types = ['hyperbolic']

model, avg_loss = np.zeros((len(dist_types),len(e_size)),dtype=object),np.zeros((len(dist_types),len(e_size)))

names = ['largest_group_strings', 'strings_test', 'strings_subset','clean_strings']

for name in names:
    dataset_name = '\content\' + name+'.pkl'

    for i in range(len(dist_types)):
        for j in range(len(e_size)):

            print(dist_types[i])

            model[i][j], avg_loss[i][j] = run_model(dataset_name,e_size[j],dist_types[i],string_size,n_epoch)

    pickle.dump((model,avg_loss,e_size,dist_types), open('\content\'+name+'.pkl', "wb"))



# Analysis
## Synthetic Dataset Results

Please access the figures at these links below!


https://github.com/nongiga/NeuroSEED/blob/master/Figure%202A.png
Figure 2A Comparison of Loss over Embedding Dimensions - Strings_Test

https://github.com/nongiga/NeuroSEED/blob/master/Figure%202B.png
Figure 2B Raw MSE Loss Data for Each Embedding Model



The data that we collected demonstrates a much lower loss for hyperbolic embedding than Euclidean embeddings across all embedding dimensions. Hyperbolic embeddings also demonstrate lower loss than all other commonly used methods of embedding aside from embedding using cosine distance for dimensions less than or equal to 4. Hyperbolic embeddings outcompete all other embedding models for higher dimensional embeddings. Further research is needed to explain the difference in using cosine embedding models.

This builds upon previous NeuroSEED research (Corso et al, 2021) which does not contain the discrepancy in cosine embeddings in low dimensions. More research is needed to explain why this occurs in the specific instance of genetic sequence interpretation.

### Benchmark Comparison

https://github.com/nongiga/NeuroSEED/blob/master/Figure%203.png
Figure 3: Comparison of Average Losses Between Embeddings compared to Levenshtein Distances

Standard models of embedding for genetic strings use Euclidean distance. Here, we illustrate that Euclidean models fare far worse than hyperbolic embeddings in accurately estimating the distance between strings. We also provide comparisons for other commonly used distances. Interestingly, Manhattan distances perform remarkably well. One explanation is that the number of mutations necessary to move from one smORF to the next can be closely approximated with a Manhattan-like calculation (e.g. AAA to TTT has a distance of 3 Manhattan-wise and requires at minimum 3 mutations to do so). Further verification is necessary to explain this phenomenon fully.


#Real-World Applications

## smORF Discovery

There are a litany of smORFs that have yet to be properly classified. smORFs comprise an enormous component of cellular function, and play a vital role in nearly every major process, but remain under-targeted in research because it is very difficult to classify and study their function. Our method provides a more efficient path to classifying smORFs.

## Hyperbolic Embedding Framework
This framework also provides the groundwork for a method of comparing non-smORF proteins. In particular, our embedding method reveals the ability to partially reconstruct ancestors of existing smORF sequences and understand which mutations exist now. This provides a window into the past where we can understand how small mutations may have impacted the function of an ancestral smORF to give rise to the present diversity within a class of smORFs. More broadly, we can extend this power for longer proteins, magnifying efficiency gains compared to traditional Euclidean techniques. In doing so, we give rise to predictive power in synthesizing artificial biological tools in research and in clinic. Furthermore, our hyperbolic embedding framework provides a general power capacity to compare other hierarchical data beyond genetic sequences more efficiently.

# Limitations
A keras tuner-like function that automatically samples across hyperparameters and learning models would be beneficial to test across a variety of models during experimentation.

Furthermore, we would have liked to make the encoder model less of a black box. One key result of the model is the possibility to understand ancestral relationships within clusters (e.g. region X in any sequence is more prone to A → C mutation than an A → T at position x1). These relationships can be hidden within the encoder within the weights between locally grouped neurons. The understanding of which particular regions are more likely to be conserved and which are more likely to be mutated in which ways is a key result that would be beneficial to the implications of the projects.

Given the quantity of smORF data, we would have liked to use some stronger form of Recurrent Neural Network to take advantage of memory gains. For example, past research has demonstrated that LSTM networks are more effective at learning large quantities of time-series EEG data. Although the structure of such data is not hierarchical, the patterns of brain waves being determined by neighboring patterns can be similarly modeled by our embedding model. Consequently, we expect that using an LSTM architecture may advance the accuracy of our model.

In the future, we hope to adapt this model to learn directly from amino acid mutations, and demonstrate that the encoder learns on amino acids, in order to further validate our results. 


# References
Chamberlain, B.P., Clough, J.R., & Deisenroth, M.P. (2017). Neural Embeddings of Graphs in Hyperbolic Space. ArXiv, abs/1705.10359.

Corso, Gabriele et al. “Neural Distance Embeddings for Biological Sequences.” ArXiv abs/2109.09740 (2021): n. Pag.

Durrant MG, Bhatt AS. Automated Prediction and Annotation of Small Open Reading Frames in Microbial Genomes. Cell Host Microbe. 2021;29: 121–131.e4. Pmid:33290720

Gray T, Storz G, Papenfort K. Small Proteins; Big Questions. J Bacteriol. 2022 Jan 18;204(1):e0034121. doi: 10.1128/JB.00341-21. Epub 2021 Jul 26. PMID: 34309401; PMCID: PMC8765408.

H. Sberro, B.J. Fremin, S. Zlitni, F. Edfors, N. Greenfield, M.P. Snyder, G.A. Pavlopoulos, N.C. Kyrpides, A.S. Bhatt
Large-scale analyses of human microbiomes reveal thousands of small, novel genes
Cell, 178 (2019), pp. 1245-1259.e14

